# Introduction to Python for Earth Observation Science

This notebook contains exercises, demonstrating fundamental principles of Python and Software Egineering in general to work effectively with Earth Observation data and processes.

We will explain concepts like:
- What is a datacube and why is it important
- Various ways a datacube can be represented in storage and how to load them
- Investigating and visualising datacubes
- The filter-, map-, reduce-pattern and its relevants to EO data processing and analysis
- A first use case calculating monthly soil moisture means, illustrating a typical EO scientist's workflow
- The convolution operator and how it is implemented in xarray

Further down the notebook, the astute student can look into advanced concepts like:
- Data chunking and its impact on performance
- Using dask to scale up processes

## What is a datacube and why should I care?

Explain the basic concept of a datacube, and why it is important

## Typical ways a datacube is represented

- Stack of files, e.g. GeoTIFFs NetCDF files
- Cloud storage optimised, e.g. Zarr archive
- Can be on simple block storage (hard disk) or cloud storage (e.g. Amazon S3 bucket)

### Loading a datacube from a stack of NetCDF files using xarray

In our example, we will use 2 months from the CCI active SSM which is delivered as a stack of NetCDF.
This setup is very typical.
The CCI SSM is delivered daily covers the entire globe with a resolution of X.
Some more dataset descriptions.
The file stack is downloaded and prepared on this notbook in the data folder, with the same naming and path conventions as it is deliverd from the Copernicus service.

We will use xarray to open and investigate the datacube.

In [16]:
import numpy as np
# We import the necessary libraries for the upcoming steps
import xarray as xr
from pathlib import Path

# The data root where all NetCDFs are stored following the Copernicus CCI path convention
data_root = Path('data/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2022/')

xarray's `open_mfdataset` provides a convenient means of loading a datacube represented as a stack of files.
It is assumed, that these files are already co-registers, meaning they are in the same projection and resolution.
We will discuss reprojection and co-registering data in one of the following exercises.

In [2]:
# open_mfdataset takes a sequence of files describing the datacube
# it is also possible to specify the data path as a string with a wildcard, e.g.:
# "path/to/files/*.nc" - you can try it out
cci_ssm_datacube = xr.open_mfdataset(data_root.glob("*.nc"))
cci_ssm_datacube

<xarray.Dataset> Size: 2GB
Dimensions:         (time: 62, lat: 720, lon: 1440)
Coordinates:
  * time            (time) datetime64[ns] 496B 2022-07-01 ... 2022-08-31
  * lat             (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon             (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
Data variables:
    sm              (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sm_uncertainty  (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    flag            (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    freqbandID      (time, lat, lon) int16 129MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    dnflag          (time, lat, lon) int8 64MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mode            (time, lat, lon) int8 64MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sensor          (time, lat, lon) int32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    t0              (time, lat, lon) datetime64[ns] 514MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes: (12/44)
    title:                        ESA CCI Surface Soil Moisture merged ACTIVE...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL), Planet ...
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    geospatial_lon_units:         degrees_east
    geospatial_lat_resolution:    0.25 degree
    geospatial_lon_resolution:    0.25 degree
    spatial_resolution:           25km
    time_coverage_start:          20220701T000000Z
    time_coverage_end:            20220701T235959Z

xarray provides us with information on the data contained within the datacube, as well as a platora of metadata.
You can see that it contains multiple data variables, but for now we are only interested in soil moisture:

In [4]:
ssm_datacube = cci_ssm_datacube['sm']
ssm_datacube

<xarray.DataArray 'sm' (time: 62, lat: 720, lon: 1440)> Size: 257MB
dask.array<concatenate, shape=(62, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 496B 2022-07-01 2022-07-02 ... 2022-08-31
  * lat      (lat) float64 6kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Attributes:
    _CoordinateAxes:      time lat lon
    ancillary_variables:  sm_uncertainty flag t0
    dtype:                float32
    long_name:            Percent of Saturation Soil Moisture
    units:                percent
    valid_range:          [  0. 100.]

The array displayed here already contains a lot of information, and we will talk about the most important ones:

- **Dimensions:** Our datacube has 3 dimensions, _time_, _lat_ and _lon_. In this case we talk about a _spatio temporal_ datacube.
- **Coordinates:** We can see that each demension is associated with coordinated values. The time coordinates are represented as datetime objects, and spatial coordinates in lat/lon degrees.
- **valid range and units:** The `valid_range` and `units` attributes tell us that soil moisture is given in relative units, represented as percentages from 0 to 100.
- **Chunk**: This tells us that the datacube is chunked spatially, meaning that each file covers the entire globe but only for one day. This will become very important in the chapter chunking. 

### Quick investigation of datacube

Once the data is loaded, it is usually a good idea to plot them, making sure they have been loaded correctly, and get a rough understanding of it.
Plotting spatial and temporal.
Short intro to cmaps?

#### Spatial plotting

First, we will do a spatial plot of one of the datacube slices, to get an idea of the spatial extent of our data.
xarray already provides some nice standard plotting mechanisms using matplotlib which we can utilise immediately.
They are not very sophisticated but perfect to get a feel for the data.

In [5]:
import matplotlib.pyplot as plt
%matplotlib notebook

# we will create individual figures for each plot, otherwise subsequent plots overwrite this when using the interactive mode
plt.figure() 
# we will simply select the first slice using `[0]` in the timeseries, usually a good starting point
ssm_datacube[0].plot.imshow()

<IPython.core.display.Javascript object>

Not the `%matplotlib notebook` magic command. By invoking this line tells matplotlib that we are in a jupyer notebook environment, and it will create an interactive plot for us.

Since xarray handles all the coordinates, labels and metadata of our datacube for us, we already get a quite nice plot out of the box containing sensible labels and colour bars.

The title tells us the time of data slice, and we see the lat/lon extent covering almost the entire globe.
Furthermore, the colour bar also tells us the proper units of our datacube values.

Because we used an interactive plot, we can see the lat/lon as well as the precise value when we hover over the map.
This will become very useful for the next step.

#### Temporal plotting

Using the interactive plot above, we can select some lat/lon coordinates for which we can plot the time series of our data.
This helps us in choosing a location that actually contains data, and not for instance some location in the middle of the sea.

xarray provides the `sel` method for us to select points based on coordinates. 

In [6]:
# with the sel method we can select data based on the coordinates of the datacube
# when we specify `method='nearest'` we don't need to select the precise lat/lon coordinate, but xarray will find the closest one for us
plt.figure() 
ssm_datacube.sel(lon=25, lat=50, method='nearest').plot()

<IPython.core.display.Javascript object>

Notice that in the previous section before we used index based selection instead of the actual coordinates.
This means that the index is specified by the actual grid cells instead of using a coordinate value **associated** with the grid cell.

For spatial indices, this means that each pixel has its own lat/lon coordinate associated with it.
Consequently in the _time_ dimension each cell is also associated with a datetime.

**Exercise**: How would you select a spatial slice of the data using a date instead of an index like we did in the previous section?

In [7]:
# Hint: python has a `datetime` object (`from datetime import datetime`) to represent timestamps

Notice also that plotting the time series seems to take a bit longer than plotting an entire global slice.
You can speculate about the reason for this now, but we will talk about what might cause this in the upcoming advanced topic.

**Bonus:** You can measure execution time using the `%timeit` magic command. It will run the statement after it a couple of times and measure execution time.
For example, we can measure how long it takes to load a temporal slice and accessing all its values via the `values` property:

In [6]:
%timeit ssm_datacube[:, 200, 800].values

35.7 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notice that we didn't plot anything, because the rendering part of the plot could interfere with measuring the data access, the operation we are actually interested.
We also used direct indices instead of `sel` to make sure we don't measure coordinate lookup. 
The `:` index means that we want to access all values along this axis.
Now try measuring accessing a spacial slice, like in the first plot we did.

6.15 ms ± 274 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Did you see any significant differences?

## Operating on datacubes

When data is properly aligned it is much easier to perform operations on a datacube.
There are many common operations "building blocks" most analysis and algorithms can be broken down to:
- filtering: selecting specific data, can be temporal or spetial or both
- map: transforming (selected) data in a certain way, for example scaling or normalising data
- reduction: aggregating many datapoints to a single datapoint, for example, mean, min/max, percentiles etc.

With a datacube it is easy to perform these operations.
Most algorithms can be broken down into these princinples.
Give examples: ...



### Use-case: Calculate monthly mean over region of interest

In this use-case we will calculate the monthly mean over Europe using the filter-, map-, reduce-pattern.
It will illustrate a common workflow of EO scientists to manipulate and analyse data.

**Filter:** We perform a spatial filter operation to select our region of interest (ROI). 

Tip: You can use the rectangle selection tool in the interactive plot above to find out the lon/lat bounding box of europe.

In [8]:
# by passing `slice` objects as lon/lat coordinates we can specify a range
# there is also an error in this selection, can you spot it?
ssm_eu = ssm_datacube.sel(lon=slice(-10, 45), lat=slice(30, 70))    

# now plot again the first slice, making sure we selected the correct ROI
plt.figure()
ssm_eu[0].plot.imshow()

<IPython.core.display.Javascript object>

Pro-Tip: Cache region of interest to speed up development process
We use Zarr, because _list zarr advantages and talk about data formats in general_

In [9]:
import tempfile

# `tempfile.mkdtemp` creates a temporary directory for us with write access
eu_cache = Path(tempfile.mkdtemp()) / "eu_cache.zarr"
ssm_eu.to_zarr(eu_cache)

Now let's load the cached zarr archive again, and visualise some parts making sure everything worked as expected:

In [10]:
# xarray `DataArray`s will always be converted to `Datasets` when stored as zarr archive
# this is why we need to select the `DataAray` again via `['sm']`
ssm_eu = xr.open_zarr(eu_cache)['sm']

# plot the ROI again making sure caching works as expected
plt.figure()
ssm_eu[0].plot.imshow()

<IPython.core.display.Javascript object>

**Map:** Nothing to do in the case of simple means, i.e. its the identy operator.

**Reduce**: Now let's calculate the mean of the data spanning one month. Luckily xarray comes with a `mean` shorthand, which we can use. The only thing we need to specify is along which axis we want to apply the mean reduction. Because we are interested in the average soil moisture of one month, it is the `time` dimension.

In [15]:
ssm_mean_eu = ssm_eu.mean(dim='time')

plt.figure()
# notice that we don't have to select a time index
# the time dimension has been collapsed by the `mean` reduction operation
ssm_mean_eu.plot.imshow()

<IPython.core.display.Javascript object>

This `mean` function exists for convenience, but we can achieve same thing use the more flexible `reduce` function.
We will demonstrate this here, since the reduce function will become important in later exercises.

In [18]:
# we are now reimplementing the `mean` function using xarray's `reduce` operator.
# the `reduce` operator hands the xarray data as numpy array and the axis to reduce to the specified function.
# this function then needs to collapse the array along the specified axis.
def mean_custom(array_data, axis):
    # the mean implementation is trivial, since we just hand array and axis over to numpy's mean operator
    return np.nanmean(array_data, axis=axis)

ssm_mean_eu_from_reduce = ssm_eu.reduce(mean_custom, 'time')

plt.figure()
ssm_mean_eu_from_reduce.plot.imshow()

<IPython.core.display.Javascript object>

### What about weakly means?

### The importance of convolutions

Special mentioning: convolutions. 
Rolling window, the so called, kernel, across time and/or space with stride.
Common special edge case is upsampling or coarsening of data: A mean kernel with stride length equal 

Put nice convolution operation here explaining kernel size and stride.

Show example coarsening of Mozambique data

## Exercise: Smooth soil moisture using rolling mean

Apply what learned and calculate a rolling mean over Mosambique (lat lon bbox).
Hint: the `rolling` function of xarray

## Advanced: Chunking and operation scaling

Look at the importance of chunking data.
Dask/ zarr in combination, allows us to store data in differently aligned chunks.
What does that mean? -> What is easier to load, speak about eager munching and cache loading etc.

Demonstrate the problem of misaligned chunks here with performing mean operation and timing it.

Also NetCDF can store chunked data, however zarr is recomended, because more modern file format


If data properly chunked and operation trivially parallelisable we can use dask for scaling.

Show example of using dask scheduler for scaling operations

Explain graph and how it works

### Exercise: Speed up the rolling mean over Mosambique